In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
from scipy.io import loadmat
from skimage.measure import regionprops
from skimage.io import imshow
from matplotlib.colors import LinearSegmentedColormap
from collections import defaultdict
import shutil

In [ ]:
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/Pascal_dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
Classes in VOC 2010
1: aeroplane, 2: bicycle, 3: bird, 4: boat, 5: bottle, 6: bus,
7: car, 8: cat, 9: chair, 10: cow, 11: table, 12: dog,
13: horse, 14: motorbike, 15: person, 16: pottedplant,
17: sheep, 18: sofa, 19: train, 20: tvmonitor
'''

'\nClasses in VOC 2010\n1: aeroplane, 2: bicycle, 3: bird, 4: boat, 5: bottle, 6: bus,\n7: car, 8: cat, 9: chair, 10: cow, 11: table, 12: dog,\n13: horse, 14: motorbike, 15: person, 16: pottedplant,\n17: sheep, 18: sofa, 19: train, 20: tvmonitor\n'

In [ ]:
# Pascal VOC color map (official format)
def color_map(N=256, normalized=True):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << (7 - j))
            g = g | (bitget(c, 1) << (7 - j))
            b = b | (bitget(c, 2) << (7 - j))
            c = c >> 3
        cmap[i] = np.array([r, g, b])
    cmap = cmap / 255 if normalized else cmap
    return LinearSegmentedColormap.from_list('voc_cmap', cmap)

In [ ]:
# Load Pascal VOC color map
VOC_CMAP = color_map(21)

In [ ]:
def make_label(image_path, anno_path):
    try:
        image = plt.imread(image_path)
    except FileNotFoundError:
        print(f"Warning: Image not found {image_path}")
        return 0, 0

    image_size = image.shape
    shape = image_size[:-1]

    try:
        mat_data = loadmat(anno_path)['anno'][0, 0]
    except FileNotFoundError:
        print(f"Warning: Annotation not found {anno_path}")
        return 0, 0

    n_objects = mat_data['objects'].shape[1]
    mask = np.zeros(shape, dtype=np.uint8)
    found = False

    for obj in mat_data['objects'][0, :]:
        class_ind = obj['class_ind'][0, 0]

        # Mark all pixels belonging to this object’s mask as its class index
        if 'mask' in obj.dtype.names:
            obj_mask = obj['mask']
            mask[obj_mask > 0] = class_ind
            found = True

        # If it has parts, include them too (optional)
        if obj['parts'].shape[1] > 0:
            for part in obj['parts'][0, :]:
                if 'mask' in part.dtype.names:
                    part_mask = part['mask']
                    mask[part_mask > 0] = class_ind
                    found = True

    return (image_size, mask) if found else (0, 0)

In [ ]:
def main():
    print(f"Using Base Directory: {BASE_DIR}")

    mat_dir = os.path.join(BASE_DIR, 'Annotations_Part/')
    origimage_dir = os.path.join(BASE_DIR, 'JPEGImages/')
    label_dir = os.path.join(BASE_DIR, 'masks_colored/')
    image_dir = os.path.join(BASE_DIR, 'images/')

    os.makedirs(label_dir, exist_ok=True)
    os.makedirs(image_dir, exist_ok=True)

    list_mats = os.listdir(mat_dir)
    print(f"Found {len(list_mats)} annotation files to process.")

    processed = skipped = 0

    for i, mat in enumerate(list_mats):
        if i % 100 == 0 and i > 0:
            print(f"Processed {i}/{len(list_mats)} files...")

        imname = mat[:-4] + '.jpg'
        image_path = os.path.join(origimage_dir, imname)
        mat_path = os.path.join(mat_dir, mat)

        image_size, mask = make_label(image_path, mat_path)
        if image_size == 0:
            skipped += 1
            continue

        processed += 1
        shutil.copy(image_path, image_dir)

        # Save mask with unique color per class
        mpl.rcParams['savefig.pad_inches'] = 0
        figsize = (image_size[1] / 100, image_size[0] / 100)
        fig = plt.figure(figsize=figsize)
        ax = plt.axes([0, 0, 1, 1], frameon=False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        plt.autoscale(tight=True)

        max_label = np.max(mask)
        custom_cmap = color_map(N=max_label + 1)
        ax.imshow(mask, cmap=custom_cmap, vmin=0, vmax=max_label)

        save_path = os.path.join(label_dir, imname.replace('.jpg', '.png'))
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

    print("\n--- Processing Complete ---")
    print(f"Total processed: {processed}")
    print(f"Total skipped (no objects found): {skipped}")
    print(f"Colored masks saved to: {label_dir}")

In [ ]:
main()

Using Base Directory: /content/drive/MyDrive/Pascal_dataset/
Found 10103 annotation files to process.
Processed 100/10103 files...
Processed 200/10103 files...
Processed 300/10103 files...
Processed 400/10103 files...
Processed 500/10103 files...
Processed 600/10103 files...
Processed 700/10103 files...
Processed 800/10103 files...
Processed 900/10103 files...
Processed 1000/10103 files...
Processed 1100/10103 files...
Processed 1200/10103 files...
Processed 1300/10103 files...
Processed 1400/10103 files...
Processed 1500/10103 files...
Processed 1600/10103 files...
Processed 1700/10103 files...
Processed 1800/10103 files...
Processed 1900/10103 files...
Processed 2000/10103 files...
Processed 2100/10103 files...
Processed 2200/10103 files...
Processed 2300/10103 files...
Processed 2400/10103 files...
Processed 2500/10103 files...
Processed 2600/10103 files...
Processed 2700/10103 files...
Processed 2800/10103 files...
Processed 2900/10103 files...
Processed 3000/10103 files...
Process